In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Assume there are 100 drivers making decisions about where to go
num_drivers = 100

# Extend the kelurahan_data with additional trip and driver characteristics
np.random.seed(42)  # For reproducibility

# Adding simulated driver characteristics
driver_data = pd.DataFrame({
    'DriverID': np.arange(1, num_drivers + 1),
    'DrivingExperience': np.random.uniform(1, 10, num_drivers),  # Years of driving experience
    'Preference': np.random.choice(['Speed', 'Safety', 'Earnings'], num_drivers),  # Driver's preference
    'HistoricalFrequentKelurahan': np.random.choice(kelurahan_data['Kelurahan'], num_drivers)  # Frequently visited kelurahan
})

# Extend the simulated kelurahan_data with trip characteristics
trip_data = pd.DataFrame({
    'TripID': np.arange(1, num_drivers + 1),
    'TripDuration': np.random.uniform(5, 60, num_drivers),  # Trip duration in minutes
    'TripDistance': np.random.uniform(1, 20, num_drivers),  # Trip distance in km
    'PeakHour': np.random.choice([0, 1], num_drivers),  # Whether the trip is during peak hour
    'TripType': np.random.choice(['Person', 'Parcel'], num_drivers)  # Type of trip
})

# Merge the driver data with trip data for a complete dataset
complete_data = pd.merge(driver_data, trip_data, left_on='DriverID', right_on='TripID')
complete_data = complete_data.join(kelurahan_data.set_index('Kelurahan'), on='HistoricalFrequentKelurahan')

# Display the first few rows of the complete data
complete_data.head()


NameError: name 'kelurahan_data' is not defined

In [7]:
# Define the complex utility function including driver and trip characteristics
def complex_utility_function(params, data):
    (b_accessibility, b_demand_density, b_competition, b_expected_profit,
     b_experience, b_duration, b_distance, b_peak_hour, b_trip_type_person) = params
    
    # Calculate utility based on kelurahan characteristics
    utility = (b_accessibility * data['Aksesibilitas'] +
               b_demand_density * data['KepadatanPermintaan'] -
               b_competition * data['Persaingan'] +
               b_expected_profit * data['KemungkinanKeuntungan'])
    
    # Add driver characteristics to utility
    utility += b_experience * data['DrivingExperience']
    
    # Add trip characteristics to utility
    utility += (b_duration * data['TripDuration'] +
                b_distance * data['TripDistance'] -
                b_peak_hour * data['PeakHour'])
    
    # Add a binary variable for trip type (1 if Person, 0 if Parcel)
    utility += b_trip_type_person * (data['TripType'] == 'Person')
    
    # Scaling utilities to avoid very large or very small values
    utility = utility - utility.max()
    
    return utility

# Define the log-likelihood function for the complex model
def complex_log_likelihood(params, data):
    utilities = complex_utility_function(params, data)
    exp_utilities = np.exp(utilities)
    probabilities = exp_utilities / exp_utilities.sum()
    probabilities = np.clip(probabilities, 1e-8, 1-1e-8)  # Avoid log(0)
    log_prob = np.log(probabilities)
    return -log_prob.mean()  # Negative of the mean log likelihood

# Initial parameter values for the complex model
initial_params_complex = [0.1] * 9  # Initial guesses for all parameters

# Perform the optimization using the Nelder-Mead method for the complex model
result_complex = minimize(complex_log_likelihood, initial_params_complex, args=(complete_data,), method='Nelder-Mead')

# Display the optimization results for the complex model
result_complex

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 5.571230173443072
             x: [ 1.760e-01  3.088e-03  1.368e-01 -1.734e-05  2.214e-01
                  3.047e-02  1.775e-01  7.459e-02 -2.142e-03]
           nit: 629
          nfev: 973
 final_simplex: (array([[ 1.760e-01,  3.088e-03, ...,  7.459e-02,
                        -2.142e-03],
                       [ 1.760e-01,  3.087e-03, ...,  7.461e-02,
                        -2.084e-03],
                       ...,
                       [ 1.760e-01,  3.055e-03, ...,  7.458e-02,
                        -2.152e-03],
                       [ 1.760e-01,  3.058e-03, ...,  7.456e-02,
                        -2.192e-03]]), array([ 5.571e+00,  5.571e+00,  5.571e+00,  5.571e+00,
                        5.571e+00,  5.571e+00,  5.571e+00,  5.571e+00,
                        5.571e+00,  5.571e+00]))